In [106]:
from pynq import Overlay
import pynq.lib.dma
overlay = Overlay('V_4_bitstream.bit')


In [107]:
import os
import cv2

os.makedirs('images', exist_ok=True)
os.makedirs('outimages', exist_ok=True)
for i in range(1):
    original_filename = f"webcam_{i}.jpeg"
    resized_filename = f"webcam_{i}_resized.jpeg"
    os.system(f"fswebcam images/{original_filename}")

    image = cv2.imread(f"images/{original_filename}")


    if image is not None:
    
        resized_image = cv2.resize(image, (64, 64))

        cv2.imwrite(f"images/{resized_filename}", resized_image)
        print(f"Image resized and saved as '{resized_filename}'")
    else:
        print(f"Error: Image '{original_filename}' not found or unable to read.")


--- Opening /dev/video0...
Trying source module v4l2...
/dev/video0 opened.
No input was specified, using the first.
Adjusting resolution from 384x288 to 352x288.
--- Capturing frame...
Captured frame in 0.00 seconds.


Image resized and saved as 'webcam_0_resized.jpeg'


--- Processing captured image...
Writing JPEG image to 'images/webcam_0.jpeg'.


In [108]:
overlay.download()

In [109]:
overlay.hierarchy_dict
for i in overlay.ip_dict:
    print(i)

axi_dma_0
FSRCNN_0
processing_system7_0


In [110]:
dma = overlay.axi_dma_0

In [111]:
import cv2
import numpy as np
in_img = cv2.imread('images/webcam_0_resized.jpeg')
height, width, nr_of_channels = in_img.shape
print('Input image size: ',in_img.shape)
print("height = ", height)

Input image size:  (64, 64, 3)
height =  64


In [112]:
#write the constants to the respective addresses
FSRCNN_ip = overlay.FSRCNN_0
scale_factor = 2
FSRCNN_ip.write(0x10,height)                            
FSRCNN_ip.write(0x18,width)  
FSRCNN_ip.write(0x20,scale_factor)                                                  
#verif_input_width = FSRCNN_ip.read(0x18)
verif_scale_factor = FSRCNN_ip.read(0x20)


In [113]:
from   pynq import allocate
import time
out_height = scale_factor*height
out_width  = scale_factor*width
size_stream_in    = height*width
size_stream_out   = out_height*out_width
red_channel_in    = allocate(shape=(size_stream_in,1),  dtype=np.uint32)
red_channel_out   = allocate(shape=(size_stream_out,1), dtype=np.uint32)
green_channel_in  = allocate(shape=(size_stream_in,1),  dtype=np.uint32)
green_channel_out = allocate(shape=(size_stream_out,1), dtype=np.uint32)
blue_channel_in   = allocate(shape=(size_stream_in,1),  dtype=np.uint32)
blue_channel_out  = allocate(shape=(size_stream_out,1), dtype=np.uint32)
k= 0
for i in range(height):
    for j in range(width):
            red_channel_in[k,0]   = in_img[i,j,0]
            green_channel_in[k,0] = in_img[i,j,1]
            blue_channel_in[k,0]  = in_img[i,j,2]
            k = k + 1

In [114]:
import time
from contextlib import suppress
start_time = time.time()
# Trigger the DMA transfer and wait for the result
dma.sendchannel.transfer(red_channel_in)  #python - dma - accelerator PYNQ
dma.recvchannel.transfer(red_channel_out) #PYNQ accelerator - dma - python
dma.sendchannel.wait()                    #makes sure that does not move on until the transfer is effectuated
stop_time = time.time()
with suppress(RuntimeError): 
    dma.recvchannel.wait()
red_channel_hw_exec_time = stop_time - start_time

KeyboardInterrupt: 

In [ ]:
overlay = Overlay('V_4_bitstream.bit')
# Load the DMA
dma = overlay.axi_dma_0 
# Load FSRCNN IP
FSRCNN_ip = overlay.FSRCNN_0

In [ ]:
start_time = time.time()
# Trigger the DMA transfer and wait for the result
dma.sendchannel.transfer(green_channel_in)  #python - dma - accelerator PYNQ
dma.recvchannel.transfer(green_channel_out) #PYNQ accelerator - dma - python
dma.sendchannel.wait()                      #makes sure that does not move on until the transfer is effectuated
stop_time = time.time()
with suppress(RuntimeError):
    dma.recvchannel.wait()
green_channel_hw_exec_time = stop_time - start_time

In [ ]:
overlay = Overlay('modified_mem.bit')
# Load the DMA
dma = overlay.axi_dma_0 
# Load FSRCNN IP
FSRCNN_ip = overlay.FSRCNN_0

In [ ]:
start_time = time.time()
# Trigger the DMA transfer and wait for the result
dma.sendchannel.transfer(blue_channel_in)  #python - dma - accelerator PYNQ
dma.recvchannel.transfer(blue_channel_out) #PYNQ accelerator - dma - python
dma.sendchannel.wait()                     #makes sure that does not move on until the transfer is effectuated
stop_time = time.time()
with suppress(RuntimeError):
    dma.recvchannel.wait()
blue_channel_hw_exec_time = stop_time - start_time

In [ ]:

hw_exec_time = red_channel_hw_exec_time + green_channel_hw_exec_time + blue_channel_hw_exec_time
print('Hardware FSRCNN execution time: ',hw_exec_time)

In [ ]:
out_img = np.zeros((out_height, out_width, nr_of_channels))
print('Output image size: ',out_img.shape)
k = 0
for i in range(out_height):
    for j in range(out_width):
            red_channel_bin = np.binary_repr(red_channel_out[k,0], width = 32)
            out_img[i,j,0] = int(str(red_channel_bin[24:32]), 2)
            green_channel_bin = np.binary_repr(green_channel_out[k,0], width = 32)
            out_img[i,j,1] = int(str(green_channel_bin[24:32]), 2)
            blue_channel_bin = np.binary_repr(blue_channel_out[k,0], width = 32)
            out_img[i,j,2] = int(str(blue_channel_bin[24:32]), 2)
            k = k+1
cv2.imwrite('/outimages/output_128x128{i}.jpeg', out_img)
#cv2.imshow('output_image',out_img)
#cv2.waitKey(0)